<a href="https://colab.research.google.com/github/zhestyatsky/MCL-WiC/blob/main/experiments/general_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!git clone https://github.com/zhestyatsky/MCL-WiC.git MCL-WiC
!pip install --upgrade -r MCL-WiC/requirements.txt

import sys
sys.path.insert(0, '/content/MCL-WiC')

Cloning into 'MCL-WiC'...
remote: Enumerating objects: 559, done.
remote: Counting objects: 100% (559/559), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 559 (delta 276), reused 424 (delta 160), pack-reused 0
Receiving objects: 100% (559/559), 5.14 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (276/276), done.
     |████████████████████████████████| 696kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 60.7MB/s 
     |████████████████████████████████| 1.2MB 60.3MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 829kB 55.3MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 276kB 55.2MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
     |████████████████████████████████| 2.9MB 54.2MB/s 
     |████████████████████████████████| 1.3MB 51.5MB/s 
     |████████████████████████████████| 143kB 56.4MB/s 
     |████████████████████████████████| 296kB 5

In [2]:
### THIS IS WHERE YOU SELECT THE MODEL ###
### ALL AVAILABLE MODELS ARE LISTED IN src/training/config.py ###

MODEL_DESCRIPTION = "bert-large-cos_sim-relu"

In [3]:
from src.data.dataset import get_train_val_test_loaders
from src.training.models import get_model
from src.training.trainer import get_trainer
from src.util.util import seed_everything

seed_everything(42)

train_loader, val_loader, test_loader = get_train_val_test_loaders(MODEL_DESCRIPTION)
clf = get_model(MODEL_DESCRIPTION)
trainer = get_trainer()

trainer.fit(clf, train_loader, val_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 333 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | activation     | ReLU             | 0     
5 | cos            | CosineSimilarity | 0     
----------------------------------------------------
333 M     Trainable params
0         Non-trainable params
333 M     Total params


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [ ]:
trainer.test(clf, val_loader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [ ]:
from src.training.hyperparams import find_threshold_if_needed

threshold, validation_accuracy = find_threshold_if_needed(val_loader.dataset.labels, clf.predictions_proba, MODEL_DESCRIPTION)
trainer.test(clf, test_loader)

Threshold: 0.5194678902626038, Validation Accuracy: 0.8082901835441589



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [ ]:
from src.testing.testing import get_results, write_outputs

results = get_results(clf.predictions_proba, threshold, test_loader.dataset)
outputs = write_outputs(results, MODEL_DESCRIPTION, '/content/gdrive/My Drive/mcl-wic-outputs/OLD_LIBS_', validation_accuracy, threshold)